In [1]:
import numpy as np
import pandas as pd


In [2]:
df = pd.read_csv('USvideos.csv')

In [3]:
df.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...


In [4]:
df.isna().sum()

video_id                    0
trending_date               0
title                       0
channel_title               0
category_id                 0
publish_time                0
tags                        0
views                       0
likes                       0
dislikes                    0
comment_count               0
thumbnail_link              0
comments_disabled           0
ratings_disabled            0
video_error_or_removed      0
description               570
dtype: int64

In [5]:
df.columns

Index(['video_id', 'trending_date', 'title', 'channel_title', 'category_id',
       'publish_time', 'tags', 'views', 'likes', 'dislikes', 'comment_count',
       'thumbnail_link', 'comments_disabled', 'ratings_disabled',
       'video_error_or_removed', 'description'],
      dtype='object')

In [6]:
df_no_na = df.copy().drop(['comments_disabled', 'ratings_disabled','video_error_or_removed'], axis=1)

In [7]:
import datetime
df_no_na['publish_time'] =  pd.to_datetime(df_no_na['publish_time'], format='%Y-%m-%dT%H:%M:%S.%fZ')

In [8]:
df_no_na['trending_date'] =  pd.to_datetime(df_no_na['trending_date'], format='%y.%d.%m')

In [9]:
df_after_17 = df_no_na[df_no_na['publish_time'].dt.year >2017]

In [10]:
df_final = df_after_17[(df_after_17['trending_date'] >= '2018-01-01') & (df_after_17['trending_date'] <= '2018-05-01')]

In [11]:
ids = df_final.video_id.unique()

In [12]:
to_be_removed =[]
for curr_id in ids:
    cur = df_final[df_final['video_id']==curr_id]
    for index, row in cur.iterrows():
        if str(row['trending_date']) == '2018-05-01 00:00:00':
            to_be_removed+= [curr_id]
            break    

In [13]:
len(df_final)

21480

In [14]:
for curr_id in to_be_removed:
    df_final = df_final[df_final['video_id']!= curr_id]

In [15]:
len(df_final)

20068

In [16]:
df_final.to_csv("US_final.csv",index=False,sep="\\")

In [17]:
df_us = pd.read_csv("US_final.csv",sep="\\")

In [18]:
# add url link column
df_us["video_url"] = "https://www.youtube.com/watch?v="+df_us["video_id"]

In [19]:
#clean tags
df_tags_cleaned = df_us.copy()
new_tags = []
for index, row in df_us.iterrows():
    temp_tags= '|'.join([tag.strip().strip('"').strip('“').strip('”').strip() for tag in df_us.iloc[index]["tags"].split("|")])
    new_tags += [temp_tags]
df_tags_cleaned["tags"] = new_tags

In [20]:
df_tags_cleaned = df_tags_cleaned[df_tags_cleaned["tags"]!="[none]"]

In [21]:
#add colunmns for category name and category cluster
cate_map = {43 : "Entertainment",29 : "Nonprofits & Activism",28 : "Science & Technology",27 : "Education",26 : "Howto & Style",25 : "News & Politics",24 : "Entertainment",23 : "Comedy",22 : "People & Blogs",20 : "Gaming",19 : "Travel & Events",17 : "Sports",15 : "Pets & Animals",10 : "Music",2 : "Autos & Vehicles",1 : "Film & Animation"}
cluster_map = {43 : "Entertainment",29 : "Activism",28 : "Technology",27 : "Education",26 : "Lifestyle",25 : "News",24 : "Entertainment",23 : "Entertainment",22 
: "Lifestyle",20 : "Entertainment",19 : "Lifestyle",17 : "Entertainment",15 : "Entertainment",10 : "Entertainment",2 : "Lifestyle",1 : "Entertainment"}

df_with_cate = df_tags_cleaned.copy()
df_with_cate['category_name']="none"
df_with_cate['cluster']="none"

for index, row in df_with_cate.iterrows():
    df_with_cate.at[index,"category_name"] = cate_map[df_with_cate.at[index,"category_id"]]
    df_with_cate.at[index,"cluster"] = cluster_map[df_with_cate.at[index,"category_id"]] 



In [22]:
#clean date format
import datetime
import math
def days_taken_to_trend(start_time,end_time):
    def to_datetime(date_str):
        return datetime.datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')
#     start = to_datetime(df_tags_cleaned.iloc[0]["publish_time"])
    start = to_datetime(publish_time)
    end = to_datetime(trending_time)

    delta_days = (end-start if end>start else datetime.timedelta(days=1)-(start-end)).total_seconds()/(3600*24)
    return delta_days

In [23]:
#calculate temperature
#temp_raw_score = views*0.2+likes*0.3+cemment_count*0.5
#if not first day in trending : temp_score = (temp_raw_score of today - temp_raw_score of prev day)/ delta_day
#if first day in trending : temp_score = (temp_raw_score of today)/ delta_day since publish

unique_videos= df_with_cate.video_id.unique()
df_with_temp = df_with_cate.copy()
df_with_temp['temperature'] = 0
# print(unique_videos[:2])
for curr_vid in unique_videos:
    curr_df = df_with_temp[df_with_temp["video_id"]==curr_vid]
    i = 0
    prev_index = 0
    prev_raw_score = 0
    for index, row in curr_df.iterrows():
#         curr_raw_score = row["views"]*0.2+row["likes"]*0.3+row["comment_count"]*0.5
        curr_raw_score = row["views"]
        if i==0: #first day
            publish_time = row["publish_time"]
            trending_time = row["trending_date"]
            delta_days = float(days_taken_to_trend(publish_time,trending_time))
            curr_temp = curr_raw_score/delta_days
        else:
            start_time = curr_df.loc[prev_index]["trending_date"]
            end_time = curr_df.loc[index]["trending_date"]
            delta_days = math.ceil(days_taken_to_trend(start_time,end_time))
            curr_temp = (curr_raw_score-prev_raw_score)/delta_days
            if curr_raw_score < prev_raw_score:
                curr_temp = curr_raw_score / delta_days
            
        df_with_temp.at[index,"temperature"] = round(curr_temp,2)
        
        i+=1
        prev_index = index
        prev_raw_score = curr_raw_score
    

In [24]:
#scale temperature down to [0~100]
df_with_temp_scaled = df_with_temp.copy()
temp_range = math.log(max(df_with_temp["temperature"])-min(df_with_temp["temperature"]))
for index, row in df_with_temp.iterrows():
    try:
        df_with_temp_scaled.at[index,"temperature"] = math.log(df_with_temp.at[index,"temperature"])/temp_range*100
    except:
        df_with_temp_scaled.at[index,"temperature"] = math.log(0.001)/temp_range*100

In [25]:
#like:dislike ratio
df_ratio = df_with_temp_scaled.copy()
df_ratio["ratio"] = 0
for i in df_ratio.index:
    ratio = df_ratio.at[i,"likes"]/df_ratio.at[i,"dislikes"] if df_ratio.at[i,"dislikes"]!=0 else df_ratio.at[i,"likes"]
    df_ratio.at[i,"ratio"] = ratio

In [26]:
#calculate trending length
df_trending_length = df_ratio.copy()
df_trending_length["trending_length"] = "" 
id_date_map = {}
for vid in unique_videos:
    mindate = df_trending_length[df_trending_length["video_id"]==vid]["trending_date"].min()
    id_date_map[vid] = mindate
for i in df_trending_length.index:
    days = pd.to_datetime(df_trending_length.at[i,"trending_date"])-pd.to_datetime(id_date_map[df_trending_length.at[i,"video_id"]])
    df_trending_length.at[i,"trending_length"] = str(days).split(' ')[0]

In [27]:
df_trending_length["category_name"].unique()

array(['Entertainment', 'Music', 'Gaming', 'Education', 'Sports',
       'News & Politics', 'People & Blogs', 'Howto & Style',
       'Science & Technology', 'Pets & Animals', 'Comedy',
       'Travel & Events', 'Film & Animation', 'Nonprofits & Activism',
       'Autos & Vehicles'], dtype=object)

In [28]:
#calculate trending length
money_map = {"Entertainment":2.5,"Music":2.5,"Gaming":6,"Education":3.9,'Sports':3,'News & Politics':2,
            'People & Blogs':3.4,'Howto & Style':4.5,
       'Science & Technology':3.9, 'Pets & Animals':3.9, 'Comedy':2.5,
       'Travel & Events':3.2, 'Film & Animation':3.6, 'Nonprofits & Activism':2.6,
       'Autos & Vehicles':3}
df_money = df_trending_length.copy()
df_money['income'] =0
for i in df_money.index:
     df_money.at[i,'income'] = (df_money.at[i,"views"]+df_money.at[i,"likes"]+df_money.at[i,"comment_count"]-df_money.at[i,"dislikes"])/1000*money_map[df_money.at[i,"category_name"]]
        


In [29]:
df_money.to_csv("US_final.csv",index=False,sep="\\")

In [30]:
df_money["ratio"].max()

843